In [ ]:
import pandas as pd
import numpy as np
import math 
from collections import defaultdict
from math import sqrt
from matplotlib import pyplot as plt
from matplotlib import ticker
import seaborn as sns
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from wordcloud import WordCloud
from tqdm import tqdm_notebook
from numpy import genfromtxt
import matplotlib.pyplot as plt

SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 18

plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

data = genfromtxt('../input/location/location.csv', delimiter=',')
data2 = genfromtxt('../input/location/location2.csv', delimiter=',')
data3= genfromtxt('../input/location/location3.csv', delimiter=',')
n_samples, n_features = data.shape
print(n_samples, n_features)

In [ ]:
def haversine(lat1, lon1, lat2, lon2): 
    dLat = (lat2 - lat1) * math.pi / 180.0
    dLon = (lon2 - lon1) * math.pi / 180.0
    lat1 = (lat1) * math.pi / 180.0
    lat2 = (lat2) * math.pi / 180.0
    a = (pow(math.sin(dLat / 2), 2) + 
         pow(math.sin(dLon / 2), 2) * 
             math.cos(lat1) * math.cos(lat2)); 
    rad = 6371
    c = 2 * math.asin(math.sqrt(a)) 
    return (rad * c)/100 
def manhattan(lat1, lon1, lat2, lon2):
  x=abs(lat2-lat1)
  y=abs(lon2-lon1)
  z=x+y
  return z
def eucledian(lat1, lon1, lat2, lon2):
  x=(lat2-lat1)*(lat2-lat1)
  y=(lon2-lon1)*(lon2-lon1)
  z=math.sqrt(x+y)
  return z
def time_calc(x,y):
  z=abs(x-y)
  return z

In [ ]:
result =pd.DataFrame({'point': '0',
	'time': 0,
	'Haversian': [0],
	'Manhatten': [0],
  'Eucledian': [0],
  'elapsed_time': [0],
  'elapsed_distance': [0]})

all_haversine_distance=0.0
all_manhattan_distance=0.0
all_eucledian_distance=0.0
all_time=0.0
for i in range(2,n_samples):
  if (i==2):
    initial_time=data[i][3]
    initial_lat=data[i][1]
    initial_long=data[i][2]
  elif (i==n_samples-1):
    final_time=data[i][3]
    final_lat=data[i][1]
    final_long=data[i][2]

  print("point {0} to {1}:".format(i-1,i))
  print('Time required',format(time_calc(data[i-1][3],data[i][3]),'.0f'),'minutes')
  print('Haversian Distance',format(haversine(data[i][1], data[i][2],data[i-1][1], data[i-1][2]),'.3f'),'Kilometer')
  print('Manhatten Distance',format(manhattan(data[i][1], data[i][2],data[i-1][1], data[i-1][2]),'.3f'),'Kilometer')
  print('Eucledian Distance',format(eucledian(data[i][1], data[i][2],data[i-1][1], data[i-1][2]),'.3f'),'Kilometer')
  print()
  w=i-1
  p=time_calc(data[i-1][3],data[i][3])
  q=format(haversine(data[i][1], data[i][2],data[i-1][1], data[i-1][2]),'.3f')
  r=format(manhattan(data[i][1], data[i][2],data[i-1][1], data[i-1][2]),'.3f')
  s=format(eucledian(data[i][1], data[i][2],data[i-1][1], data[i-1][2]),'.3f')
  all_time=all_time+p

  all_haversine_distance=all_haversine_distance+float(q)

  all_manhattan_distance=all_manhattan_distance+float(r)
  all_eucledian_distance=all_eucledian_distance+float(s)
  #print(p,q,r,s)
  new_row = {'point':int(w), 'time':int(p), 'Haversian':float(q), 'Manhatten':float(r), 'Eucledian':float(s), 'elapsed_time':float(all_time),'elapsed_distance':all_haversine_distance}
  result = result.append(new_row, ignore_index=True)

In [ ]:
result

In [ ]:
#overall distance
overall_time=time_calc(final_time,initial_time)
overall_haversine=haversine(final_lat,final_long,initial_lat,initial_long)
overall_manhatten=manhattan(final_lat,final_long,initial_lat,initial_long)
overall_eucledian=eucledian(final_lat,final_long,initial_lat,initial_long)
print('Overall Time Served in the Field by DSR: {0}'.format(overall_time))
print('First to Last Distance Considering Haversian Formula: {0}'.format(format(overall_haversine,'.3f')))
print('First to Last Distance Considering Manhatten Formula: {0}'.format(format(overall_manhatten,'.3f')))
print('First to Last Distance Considering Eucledian Formula: {0}'.format(format(overall_eucledian,'.3f')))

In [ ]:
print('Overall Time Served in the Field by DSR: {0}'.format(overall_time))
print('Overall Distance Considering Haversian Formula: {0}'.format(format(all_haversine_distance,'.3f')))
print('Overall Distance Considering Manhatten Formula: {0}'.format(format(all_manhattan_distance,'.3f')))
print('Overall Distance Considering Eucledian Formula: {0}'.format(format(all_eucledian_distance,'.3f')))

In [ ]:
print('Average Time Required to Reach Each Shop by DSR: {0}'.format(overall_time/(n_samples-2)))
print('Average Distance Between the Shops Considering Haversian Formula: {0}'.format(format(all_haversine_distance/(n_samples-2),'.3f')))
print('Average Distance Between the Shops Considering Manhatten Formula: {0}'.format(format(all_manhattan_distance/(n_samples-2),'.3f')))
print('Average Distance Between the Shops Considering Eucledian Formula: {0}'.format(format(all_eucledian_distance/(n_samples-2),'.3f')))

In [ ]:
dt=result
dt['Haversian']=dt['Haversian']*100
dt['Manhatten']=dt['Manhatten']*100
dt['Eucledian']=dt['Eucledian']*100
ct=dt
ct=ct.drop(columns=['elapsed_time','elapsed_distance'])
ct.plot(kind='bar',figsize=(25, 10),alpha=1,fontsize=18,stacked=False)
dt['Haversian']=dt['Haversian']/100
dt['Manhatten']=dt['Manhatten']/100
dt['Eucledian']=dt['Eucledian']/100

In [ ]:
def distance_between_coords(x1, y1, x2, y2):
    lat1, lon1 = x1,y1
    lat2, lon2 = x2,y2
    radius = 6371  # km
 
    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = radius * c
    return distance/100

def name_coords(coords):
    coord_count = 0
    for coord in coords:
        coord_count += 1
        coord.append(coord_count)
    return coords

def graph(coords):
    coords = name_coords(coords)
    graph = defaultdict(list)
    edges = {}
    for current in coords:
        for comparer in coords:
            if comparer == current:
                continue
            else:
                weight = distance_between_coords(current[0], current[1],
                                                 comparer[0], comparer[1])
                graph[current[2]].append(comparer[2])
                edges[current[2], comparer[2]] = weight
    return coords, edges
def shortest_path(node_list, edges, start):
    neighbor = 0
    unvisited = []
    visited = []
    total_weight = 0
    current_node = start
    for node in node_list:
        if node[2] == start:
            visited.append(start)
        else:
            unvisited.append(node[2])
    while unvisited:
        for index, neighbor in enumerate(unvisited):
            if index == 0:
                current_weight = edges[start, neighbor]
                current_node = neighbor
            elif edges[start, neighbor] < current_weight:
                current_weight = edges[start, neighbor]
                current_node = neighbor
        total_weight += current_weight
        unvisited.remove(current_node)
        visited.append(current_node)
    return visited, total_weight
def driver():
    coords=[]
    for i in range(1,n_samples):
      li=[]
      li.append(data[i][1])
      li.append(data[i][2])
      coords.append(li)
    print(coords)

    coords, edges = graph(coords)
    shortest_path(coords, edges, 3)
    shortest_path_taken = []
    shortest_path_weight = 0

    for index, node in enumerate(coords):
        path, weight = shortest_path(coords, edges, index + 1)
        print('--------------------------------------')
        print("Path", index + 1, "=", path)
        print("Weight =", weight)
        if index == 0:
            shortest_path_weight = weight
            shortest_path_taken = path
        elif weight < shortest_path_weight:
            shortest_path_weight = weight
            shortest_path_taken = path
    print('--------------------------------------')
    print("The shortest path to all nodes is:", shortest_path_taken)
    print("The weight of the path is:", shortest_path_weight)
driver()

In [ ]:
box =pd.DataFrame()
box1 =pd.DataFrame()

for i in range(1,n_samples-1):
  new_row = {'formula':'Haversian', 'distance':dt['Haversian'][i]}
  box = box.append(new_row, ignore_index=True)

for i in range(1,n_samples-1):
  new_row = {'formula':'Manhatten', 'distance':dt['Manhatten'][i]}
  box = box.append(new_row, ignore_index=True)
for i in range(1,n_samples-1):
  new_row = {'formula':'Eucledian', 'distance':dt['Eucledian'][i]}
  box = box.append(new_row, ignore_index=True)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
df=box
sns.boxplot( x=df["formula"], y=df["distance"], width=0.8, saturation=0.85)

plt.xlabel('Formula')
plt.ylabel('Distances')

plt.show()

In [ ]:
result

In [ ]:
plt.rcParams["figure.figsize"] = (25,8)
df=result  
plt.plot(df['elapsed_time'], df['elapsed_distance'], color='red', marker='o')
plt.xlabel('Time', fontsize=18)
plt.ylabel('Distance Covered', fontsize=18)
plt.grid(True)
plt.show()


#**MAP**

In [ ]:
import folium
import pandas

data = pandas.read_csv("../input/location/location.csv")


lat = list(data["lat"])
lon = list(data["long"])


map = folium.Map(location=[23.828335, 90.364255], zoom_start=6)
fg = folium.FeatureGroup(name="Bus Route")

cc = ['red', 'darkgreen', 'gray', 'green', 'blue', 'lightgreen', 'pink', 'lightblue', 'cadetblue', 'orange', 'white', 'lightgray',
       'black', 'purple', 'lightred', 'darkpurple', 'darkblue', 'darkred', 'beige']

for lt, ln in zip(lat,lon):        
    fg.add_child(folium.Marker(location=[lt, ln],popup=str(lt)+'  car',icon=folium.Icon(color='red')))
map.add_child(fg)
map.save("output.html")

In [ ]:
!pip install geocoder
import folium
from folium import plugins
import ipywidgets
import geocoder
import geopy
import numpy as np
import pandas as pd
data = pd.read_csv("../input/location/location.csv")
data

In [ ]:
# multiple markers using dataframe
# there is an example below using apply function instead of loop

# create map
map_robi = folium.Map(location=[23.1535	,90.7595], zoom_start=10)
i=1
# plot airport locations
for (index, row) in data.iterrows():
    folium.Marker(location=[row.loc['lat'], row.loc['long']], 
                  popup='shop:'+str(row.loc['sr']) , 
                  tooltip='click').add_to(map_robi)
    
# display map    
map_robi

In [ ]:
import math
from collections import defaultdict
from math import sqrt

# Shortest path to all coordinates from any node
# Coordinates must be provided as a list containing lists of
# x/y pairs. ie [[23.2321, 58.3123], [x.xxx, y.yyy]]


def distance_between_coords(x1, y1, x2, y2):
    lat1, lon1 = x1,y1
    lat2, lon2 = x2,y2
    radius = 6371  # km
 
    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = radius * c
    return distance


# Adds "names" to coordinates to use as keys for edge detection
def name_coords(coords):
    coord_count = 0
    for coord in coords:
        coord_count += 1
        coord.append(coord_count)
    return coords


# Creates a weighted and undirected graph
# Returns named coordinates and their connected edges as a dictonary
def graph(coords):
    coords = name_coords(coords)
    graph = defaultdict(list)
    edges = {}
    for current in coords:
        for comparer in coords:
            if comparer == current:
                continue
            else:
                weight = distance_between_coords(current[0], current[1],
                                                 comparer[0], comparer[1])
                graph[current[2]].append(comparer[2])
                edges[current[2], comparer[2]] = weight
    return coords, edges


# Returns a path to all nodes with least weight as a list of names
# from a specific node
def shortest_path(node_list, edges, start):
    neighbor = 0
    unvisited = []
    visited = []
    total_weight = 0
    current_node = start
    for node in node_list:
        if node[2] == start:
            visited.append(start)
        else:
            unvisited.append(node[2])
    while unvisited:
        for index, neighbor in enumerate(unvisited):
            if index == 0:
                current_weight = edges[start, neighbor]
                current_node = neighbor
            elif edges[start, neighbor] < current_weight:
                current_weight = edges[start, neighbor]
                current_node = neighbor
        total_weight += current_weight
        unvisited.remove(current_node)
        visited.append(current_node)
    return visited, total_weight


def driver():
    coords = [[23.8103,90.4125], [23.6545,90.1125],
              [23.4565,90.7895], [23.7595,90.7595],
              [23.8525,90.8525], [23.1535,90.7595],
              [23.3212,90.7898], [23.1232,90.6512], [23.9578,90.7598], [23.6515,90.6573], [23.8596,90.7496]]
    coords, edges = graph(coords)
    shortest_path(coords, edges, 3)
    shortest_path_taken = []
    shortest_path_weight = 0

    for index, node in enumerate(coords):
        path, weight = shortest_path(coords, edges, index + 1)
        print('--------------------------------------')
        print("Path", index + 1, "=", path)
        print("Weight =", weight)
        if index == 0:
            shortest_path_weight = weight
            shortest_path_taken = path
        elif weight < shortest_path_weight:
            shortest_path_weight = weight
            shortest_path_taken = path
    print('--------------------------------------')
    print("The shortest path to all nodes is:", shortest_path_taken)
    print("The weight of the path is:", shortest_path_weight)

    return shortest_path_taken

In [ ]:
L1=[]
L1=driver()

In [ ]:
L2=[]
for i in L1:
  for (index, row) in data.iterrows():
    if i == int(row.loc['sr']):
      L2.append([row.loc['lat'], row.loc['long']])
print(L2)

In [ ]:
map_robi = folium.Map(location=[23.1535	,90.7595], zoom_start=10)
i=1
# plot airport locations
for (index, row) in data.iterrows():
    folium.Marker(location=[row.loc['lat'], row.loc['long']], 
                  popup='shop:'+str(row.loc['sr']) , 
                  tooltip='click').add_to(map_robi)

folium.PolyLine(L2).add_to(map_robi)
plugins.AntPath(L2).add_to(map_robi)

    
# display map    
map_robi

In [ ]:

result2 =pd.DataFrame({'point': '0',
	'time': 0,
	'Haversian': [0], 
	'Manhatten': [0],
  'Eucledian': [0],
  'elapsed_time': [0],
  'elapsed_distance': [0]})

all_haversine_distance2=0.0
all_manhattan_distance2=0.0
all_eucledian_distance2=0.0
all_time=0.0
for i in range(2,n_samples):
  if (i==2):
    initial_time=data2[i][3]
    initial_lat=data2[i][1]
    initial_long=data2[i][2]
  elif (i==n_samples-1):
    final_time=data2[i][3]
    final_lat=data2[i][1]
    final_long=data2[i][2]

  print("point {0} to {1}:".format(i-1,i))
  print('Time required',format(time_calc(data2[i-1][3],data2[i][3]),'.0f'),'minutes')
  print('Haversian Distance',format(haversine(data2[i][1], data2[i][2],data2[i-1][1], data2[i-1][2]),'.3f'),'Kilometer')
  print('Manhatten Distance',format(manhattan(data2[i][1], data2[i][2],data2[i-1][1], data2[i-1][2]),'.3f'),'Kilometer')
  print('Eucledian Distance',format(eucledian(data2[i][1], data2[i][2],data2[i-1][1], data2[i-1][2]),'.3f'),'Kilometer')
  print()
  w=i-1
  p=time_calc(data2[i-1][3],data2[i][3])
  q=format(haversine(data2[i][1], data2[i][2],data2[i-1][1], data2[i-1][2]),'.3f')
  r=format(manhattan(data2[i][1], data2[i][2],data2[i-1][1], data2[i-1][2]),'.3f')
  s=format(eucledian(data2[i][1], data2[i][2],data2[i-1][1], data2[i-1][2]),'.3f')
  all_time=all_time+p

  all_haversine_distance2=all_haversine_distance2+float(q)

  all_manhattan_distance2=all_manhattan_distance2+float(r)
  all_eucledian_distance2=all_eucledian_distance2+float(s)
  #print(p,q,r,s)
  new_row = {'point':int(w), 'time':int(p), 'Haversian':float(q), 'Manhatten':float(r), 'Eucledian':float(s), 'elapsed_time':float(all_time),'elapsed_distance':all_haversine_distance2}
  result2 = result2.append(new_row, ignore_index=True)

In [ ]:

result3 =pd.DataFrame({'point': '0',
	'time': 0,
	'Haversian': [0],
	'Manhatten': [0],
  'Eucledian': [0],
  'elapsed_time': [0],
  'elapsed_distance': [0]})

all_haversine_distance3=0.0
all_manhattan_distance3=0.0
all_eucledian_distance3=0.0
all_time=0.0
for i in range(2,n_samples):
  if (i==2):
    initial_time=data3[i][3]
    initial_lat=data3[i][1]
    initial_long=data3[i][2]
  elif (i==n_samples-1):
    final_time=data3[i][3]
    final_lat=data3[i][1]
    final_long=data3[i][2]

  print("point {0} to {1}:".format(i-1,i))
  print('Time required',format(time_calc(data3[i-1][3],data3[i][3]),'.0f'),'minutes')
  print('Haversian Distance',format(haversine(data3[i][1], data3[i][2],data3[i-1][1], data3[i-1][2]),'.3f'),'Kilometer')
  print('Manhatten Distance',format(manhattan(data3[i][1], data3[i][2],data3[i-1][1], data3[i-1][2]),'.3f'),'Kilometer')
  print('Eucledian Distance',format(eucledian(data3[i][1], data3[i][2],data3[i-1][1], data3[i-1][2]),'.3f'),'Kilometer')
  print()
  w=i-1
  p=time_calc(data3[i-1][3],data3[i][3])
  q=format(haversine(data3[i][1], data3[i][2],data3[i-1][1], data3[i-1][2]),'.3f')
  r=format(manhattan(data3[i][1], data3[i][2],data3[i-1][1], data3[i-1][2]),'.3f')
  s=format(eucledian(data3[i][1], data3[i][2],data3[i-1][1], data3[i-1][2]),'.3f')
  all_time=all_time+p

  all_haversine_distance3=all_haversine_distance3+float(q)

  all_manhattan_distance3=all_manhattan_distance3+float(r)
  all_eucledian_distance3=all_eucledian_distance3+float(s)
  #print(p,q,r,s)
  new_row = {'point':int(w), 'time':int(p), 'Haversian':float(q), 'Manhatten':float(r), 'Eucledian':float(s), 'elapsed_time':float(all_time),'elapsed_distance':all_haversine_distance3}
  result3 = result3.append(new_row, ignore_index=True)

In [ ]:
df=result2
plt.plot(df['elapsed_time'], df['elapsed_distance'], color='green', marker='o')
plt.xlabel('Time', fontsize=18)
plt.ylabel('Distance Covered', fontsize=18)
plt.grid(True)
plt.rcParams["figure.figsize"] = (25,8)
plt.show()

In [ ]:
df=result3
plt.plot(df['elapsed_time'], df['elapsed_distance'], color='blue', marker='o')
plt.xlabel('Time', fontsize=18)
plt.ylabel('Distance Covered', fontsize=18)
plt.grid(True)
plt.rcParams["figure.figsize"] = (25,8)
plt.show()


In [ ]:
result

In [ ]:
import matplotlib.pyplot as plt
# line 1 points
x1 = [10,20,30]
y1 = [20,40,10]
# plotting the line 1 points 
plt.plot(result['elapsed_time'], result['elapsed_distance'], color='red', marker='o',label='line 1')
plt.plot(result2['elapsed_time'], result2['elapsed_distance'], color='green', marker='o',label='line 2')
plt.plot(result3['elapsed_time'], result3['elapsed_distance'], color='blue', marker='o',label='line 3')
#plt.plot(x1, y1, label = "line 1")

plt.xlabel('Time', fontsize=18)
plt.ylabel('Distance Covered', fontsize=18)
plt.grid(True)
plt.rcParams["figure.figsize"] = (25,8)
plt.show()

In [ ]:
box1 =pd.DataFrame()
for i in range(1,n_samples-1):
  new_row = {'Person':'DSR-1', 'distance':result['Haversian'][i]}
  box1 = box1.append(new_row, ignore_index=True)
for i in range(1,n_samples-1):
  new_row = {'Person':'DSR-2', 'distance':result2['Haversian'][i]}
  box1 = box1.append(new_row, ignore_index=True)
for i in range(1,n_samples-1):
  new_row = {'Person':'DSR-3', 'distance':result3['Haversian'][i]}
  box1 = box1.append(new_row, ignore_index=True)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
df=box1
sns.boxplot( x=df["Person"], y=df["distance"], width=0.8, saturation=0.85)
plt.xlabel('Person')
plt.ylabel('Distances')
plt.grid(True)
plt.show()